# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [30]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data assesing

In [25]:
portfolio.shape

(10, 6)

In [3]:
portfolio.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [4]:
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
channels      10 non-null object
difficulty    10 non-null int64
duration      10 non-null int64
id            10 non-null object
offer_type    10 non-null object
reward        10 non-null int64
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


Channels can be expanded, so it will be easier to process it in the next step. id must be translated into more numerical categorical value to make it easy to analze.

In [5]:
portfolio.describe()

,difficulty,duration,reward
count,10.000000,10.000000,10.000000
mean,7.700000,6.500000,4.200000
std,5.831905,2.321398,3.583915
min,0.000000,3.000000,0.000000
25%,5.000000,5.000000,2.000000
50%,8.500000,7.000000,4.000000
75%,10.000000,7.000000,5.000000
max,20.000000,10.000000,10.000000


In [7]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


there are odd value in age, the oldest human in teh world has age 117. Furthermore, with income Nan, it can be infer that data with age 118 is flaw.

In [8]:
profile.shape

(17000, 5)

In [9]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
age                 17000 non-null int64
became_member_on    17000 non-null int64
gender              14825 non-null object
id                  17000 non-null object
income              14825 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.1+ KB


there are null value in gender and income but its size is small. It will not be a problem to delete incomplete data from dataset. It is not proper if became member in int, it must be changed into date.

In [22]:
profile[profile.gender.isnull()]

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN
6,118,20170925,None,8ec6ce2a7e7949b1bf142def7d0e0586,NaN
7,118,20171002,None,68617ca6246f4fbc85e91a2a49552598,NaN
9,118,20161122,None,8974fc5686fe429db53ddde067b88302,NaN
10,118,20170824,None,c4863c7985cf408faee930f111475da3,NaN
11,118,20150919,None,148adfcaa27d485b82f323aaaad036bd,NaN
17,118,20170801,None,744d603ef08c4f33af5a61c8c7628d1c,NaN
23,118,20170907,None,2b826eba31074a059d63b0ae8f50b7d5,NaN


All null gender has the same row with 118 years old age. It is enough do delete row with 118 years old age.

In [10]:
profile.describe()

,age,became_member_on,income
count,17000.000000,1.700000e+04,14825.000000
mean,62.531412,2.016703e+07,65404.991568
std,26.738580,1.167750e+04,21598.299410
min,18.000000,2.013073e+07,30000.000000
25%,45.000000,2.016053e+07,49000.000000
50%,58.000000,2.017080e+07,64000.000000
75%,73.000000,2.017123e+07,80000.000000
max,118.000000,2.018073e+07,120000.000000


In [11]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


value column need to be extracted. 

In [13]:
transcript.shape

(306534, 4)

In [12]:
transcript.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
event     306534 non-null object
person    306534 non-null object
time      306534 non-null int64
value     306534 non-null object
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [17]:
transcript.describe()

,time
count,306534.000000
mean,366.382940
std,200.326314
min,0.000000
25%,186.000000
50%,408.000000
75%,528.000000
max,714.000000


### Data assesing summary

1. expand channels column in portofolio data
2. encode id in portofolio data
3. encode id in profile data
4. delete data with age equal to 118 in profile data
5. Change became_member_on column in transcript data into datetime
6. encode id in profile data
7. extract value column in transcript data
8. encode extracted value in transcript data

## Data Cleaning

In [31]:
# copy dataframe
pf_copy = portfolio.copy()
prof_copy = profile.copy()
trans_copy = transcript.copy()

`1. expand channels column in portofolio data`

### Code

In [33]:
pf_copy.head()

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5


In [131]:
# expand df.tags into its own dataframe
tags = pf_copy['channels'].apply(pd.Series)

KeyError: 'channels'

In [79]:
# add aditional column for web, email, mobile and social
pf_copy['web'] = (tags.where(tags=='web') == tags.where(tags=='web')).sum(axis=1)
pf_copy['email'] = (tags.where(tags=='email') == tags.where(tags=='email')).sum(axis=1)
pf_copy['mobile'] = (tags.where(tags=='mobile') == tags.where(tags=='mobile')).sum(axis=1)
pf_copy['social'] = (tags.where(tags=='social') == tags.where(tags=='social')).sum(axis=1)
pf_copy.drop(columns=['channels'], inplace = True)

### Test

In [80]:
pf_copy.head()

,difficulty,duration,id,offer_type,reward,web,email,mobile,social
0,10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10,0,1,1,1
1,10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10,1,1,1,1
2,0,4,3f207df678b143eea3cee63160fa8bed,informational,0,1,1,1,0
3,5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5,1,1,1,0
4,20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5,1,1,0,0


`2. translate id in portofolio data`

### Code

In [86]:
# make dictionary of portfolio id
portfolio_id = {}

for ind, val in enumerate(list(pf_copy.id.unique())):
    portfolio_id[val] = ind+1

In [89]:
# mapping id based on dictionary
pf_copy.replace({'id':portfolio_id}, inplace = True)

### Test

In [90]:
pf_copy.head()

,difficulty,duration,id,offer_type,reward,web,email,mobile,social
0,10,7,1,bogo,10,0,1,1,1
1,10,5,2,bogo,10,1,1,1,1
2,0,4,3,informational,0,1,1,1,0
3,5,7,4,bogo,5,1,1,1,0
4,20,10,5,discount,5,1,1,0,0


`3. encode id in profile data`

### Code

In [95]:
# make dictionary of portfolio id
prof_id = {}

for ind, val in enumerate(list(prof_copy.id.unique())):
    prof_id[val] = ind+1

In [96]:
# mapping id based on dictionary
prof_copy.replace({'id':prof_id}, inplace = True)

### Test

In [102]:
prof_copy.id.unique()

array([    1,     2,     3, ..., 16998, 16999, 17000])

`4. delete data with age equal to 118 in profile data`

### Code

In [108]:
prof_copy.drop(prof_copy[prof_copy.age==118].index, inplace = True)

### Test

In [109]:
prof_copy[prof_copy.age==118]

,age,became_member_on,gender,id,income


In [114]:
prof_copy.reset_index().drop(columns=['index'])

,age,became_member_on,gender,id,income
0,55,20170715,F,2,112000.0
1,75,20170509,F,4,100000.0
2,68,20180426,M,6,70000.0
3,65,20180209,M,9,53000.0
4,58,20171111,M,13,51000.0
5,61,20170911,F,14,57000.0
6,26,20140213,M,15,46000.0
7,62,20160211,F,16,71000.0
8,49,20141113,M,17,52000.0
9,57,20171231,M,19,42000.0


`5. Change became_member_on column in transcript data into datetime`

### Code

In [115]:
prof_copy['became_member_on'] = prof_copy['became_member_on'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

### Test

In [116]:
prof_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14825 entries, 1 to 16999
Data columns (total 5 columns):
age                 14825 non-null int64
became_member_on    14825 non-null datetime64[ns]
gender              14825 non-null object
id                  14825 non-null int64
income              14825 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 694.9+ KB


In [117]:
prof_copy.head()

,age,became_member_on,gender,id,income
1,55,2017-07-15,F,2,112000.0
3,75,2017-05-09,F,4,100000.0
5,68,2018-04-26,M,6,70000.0
8,65,2018-02-09,M,9,53000.0
12,58,2017-11-11,M,13,51000.0


### Exploratory Data

In [118]:
# make copy of cleaned data
pf_clean = pf_copy.copy()
prof_clean = prof_copy.copy()
trans_clean = trans_copy.copy()

In [119]:
pf_clean.head()

,difficulty,duration,id,offer_type,reward,web,email,mobile,social
0,10,7,1,bogo,10,0,1,1,1
1,10,5,2,bogo,10,1,1,1,1
2,0,4,3,informational,0,1,1,1,0
3,5,7,4,bogo,5,1,1,1,0
4,20,10,5,discount,5,1,1,0,0


In [120]:
prof_clean.head()

,age,became_member_on,gender,id,income
1,55,2017-07-15,F,2,112000.0
3,75,2017-05-09,F,4,100000.0
5,68,2018-04-26,M,6,70000.0
8,65,2018-02-09,M,9,53000.0
12,58,2017-11-11,M,13,51000.0


In [121]:
trans_clean.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


`6. encode id in transcript data`

### Code

In [123]:
# make dictionary of portfolio id
tran_id = {}

for ind, val in enumerate(list(trans_copy.person.unique())):
    tran_id[val] = ind+1

In [126]:
# mapping id based on dictionary
trans_copy.replace({'person':tran_id}, inplace = True)

### Test

In [127]:
trans_copy.head()

,event,person,time,value
0,offer received,1,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,2,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,3,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,4,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,5,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


`7. extract data on value column in transcript data`

### Code

In [154]:
val_trans[val_trans['offer_id'].notnull()]

,offer id,amount,offer_id,reward
12658,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
12672,NaN,NaN,fafdcd668e3743c1bb461111dcafc2a4,2.0
12679,NaN,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,5.0
12692,NaN,NaN,ae264e3637204a6fb9bb56bc8210ddfd,10.0
12697,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
12717,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0
12721,NaN,NaN,2298d6c36e964ae4a3e7e9706d1fb8c2,3.0
12744,NaN,NaN,f19421c1d4aa40978ebb69ca19b0e20d,5.0
12764,NaN,NaN,2906b810c7d4411798c6938adc9daaa5,2.0
12767,NaN,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,10.0


In [150]:
# expand value column
val_trans = trans_copy['value'].apply(pd.Series)
val_trans.head()

,offer id,amount,offer_id,reward
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [147]:
# mapping offer id based on dictionary
val_trans.replace({'offer id':portfolio_id}, inplace = True)

In [151]:
val_trans.head()

,offer id,amount,offer_id,reward
0,9b98b8c7a33c4b65b9aebfe6a799e6d9,NaN,NaN,NaN
1,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,NaN,NaN
2,2906b810c7d4411798c6938adc9daaa5,NaN,NaN,NaN
3,fafdcd668e3743c1bb461111dcafc2a4,NaN,NaN,NaN
4,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,NaN,NaN


In [145]:
val_trans['offer id'].astype('int')

ValueError: Cannot convert non-finite values (NA or inf) to integer

### Test

In [143]:
val_trans.head()

,offer id,amount,offer_id,reward
0,4.0,NaN,NaN,NaN
1,5.0,NaN,NaN,NaN
2,10.0,NaN,NaN,NaN
3,7.0,NaN,NaN,NaN
4,2.0,NaN,NaN,NaN


`6. encode id in profile data`

### Code

### Test